### 1. 데이터 전처리

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

#### 1) CSV 파일 불러오기

In [ ]:
train = pd.read_csv('~/Downloads/kakr-4th-competition/train.csv')
label = train['income']

del train['income']

test = pd.read_csv('~/Downloads/kakr-4th-competition/test.csv')

In [ ]:
# 라벨 값 인코딩
label = label.map(lambda x: 1 if x == '>50K' else 0)

ID 컬럼은 행의 식별자로 필요 없는 컬럼이므로 삭제하겠습니다. 

In [ ]:
del train['id']
del test['id']

In [ ]:
tmp_train = train.copy()
tmp_test  = test.copy()

#### 2) 데이터 확인
.head(), .describe(), .info() 등의 함수로 데이터를 확인합니다. 

In [ ]:
tmp_train.head()

In [ ]:
tmp_train.info()

In [ ]:
tmp_train.describe()

In [ ]:
tmp_test.head()

#### 3) 결측치 처리
이전 태진님 강의에서 'workclass', 'occupation', 'native_country' 컬럼에 결측치가 있다는 것을 알 수 있었습니다. <br>
일반적인 결측치와 다르게 '?'로 표현되어있는 값들은 해당 컬럼의 최빈값으로 결측치 처리를 진행하겠습니다. <br>

##### 범주형 변수의 경우 가장 간단하게 최빈값으로 결측치 처리를 할 수 있지만, 다른 컬럼을 필터링해서 결측치 처리를 할 수 있습니다. ex) education_num 등

In [ ]:
has_na_columns = ['workclass', 'occupation', 'native_country']

In [ ]:
for c in has_na_columns:
    tmp_train.loc[train[c] == '?', c] = train[c].mode()[0]
    tmp_test.loc[test[c]   == '?', c] = test[c].mode()[0]

#### 4) Log 변환
capital_gain 변수와 capital_loss 변수의 분포가 한쪽으로 치우친 형태이므로 Log 변환을 통해 분포의 형태를 조정해주겠습니다.

In [ ]:
tmp_train['capital_gain'].plot.hist()

In [ ]:
tmp_train['log_capital_gain'] = train['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
tmp_test['log_capital_gain']  = test['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)

tmp_train['log_capital_gain'].plot.hist()

In [ ]:
train['capital_loss'].plot.hist()

In [ ]:
tmp_train['log_capital_loss'] = train['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
tmp_test['log_capital_loss'] = test['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)

tmp_train['log_capital_loss'].plot.hist()

In [ ]:
tmp_train = tmp_train.drop(columns=['capital_loss', 'capital_gain'])
tmp_test  = tmp_test.drop(columns=['capital_loss', 'capital_gain'])

#### 5) 데이터 쪼개기
##### 1. Train, Valid, Test Set
* Train Data : 모델을 학습하는데 사용하는 데이터 (모델이 알고 있는 학습할 데이터, 과거 데이터)
* Valid Data : 학습한 모델의 성능을 검증하는 데이터 (모델이 모르는 학습하지 않을 데이터, 모델 검증에 사용하는 데이터, 과거 데이터)
* Test Data : 학습한 모델로 예측할 데이터 (모델이 모르는 예측할 데이터, 미래 데이터)

##### 데이터 쪼개기, Train -> (Train, Valid)
- train_test_split 파라미터 
    - test_size  (float): Valid(test)의 크기의 비율을 지정
    - random_state (int): 데이터를 쪼갤 때 내부적으로 사용되는 난수 값 (해당 값을 지정하지 않으면 매번 달라집니다.)
    - shuffle     (bool): 데이터를 쪼갤 때 섞을지 유무
    - stratify   (array): Stratify란, 쪼개기 이전의 클래스 비율을 쪼개고 나서도 유지하기 위해 설정해야하는 값입니다. 클래스 라벨을 넣어주면 됩니다.
        - ex) 원본 Train 데이터의 클래스 비율이 (7:3) 이었다면, 쪼개어진 Train, Valid(test) 데이터의 클래스 비율도 (7:3)이 됩니다. 당연히 분류 데이터에서만 사용할 수 있습니다.

In [ ]:
from sklearn.model_selection import train_test_split

tmp_train, tmp_valid, y_train, y_valid = train_test_split(tmp_train, label, 
                                                          test_size=0.3,
                                                          random_state=2020,
                                                          shuffle=True,
                                                          stratify=label)

In [ ]:
# 인덱스 초기화
tmp_train = tmp_train.reset_index(drop=True)
tmp_valid = tmp_valid.reset_index(drop=True)
tmp_test  = tmp_test.reset_index(drop=True)

#### 6) 스케일링
Scikit-learn 라이브러리에 있는 Standard Scaler를 사용해서 수치형 변수들의 표준화를 진행하겠습니다.

In [ ]:
cat_columns = [c for c, t in zip(tmp_train.dtypes.index, tmp_train.dtypes) if t == 'O'] 
num_columns = [c for c in tmp_train.columns if c not in cat_columns]

print('범주형 변수: \n{}\n\n 수치형 변수: \n{}\n'.format(cat_columns, num_columns))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
tmp_train[num_columns] = scaler.fit_transform(tmp_train[num_columns])
tmp_valid[num_columns] = scaler.transform(tmp_valid[num_columns])
tmp_test[num_columns]  = scaler.transform(tmp_test[num_columns])

In [ ]:
tmp_train.describe()

In [ ]:
tmp_valid.describe()

In [ ]:
tmp_test.describe()

#### 6) 인코딩
범주형 변수를 수치형 변수로 인코딩 하겠습니다. 범주형 변수에는 Onehot Encoding을 적용합니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

tmp_all = pd.concat([tmp_train, tmp_valid, tmp_test])

ohe = OneHotEncoder(sparse=False)
ohe.fit(tmp_all[cat_columns])

In [ ]:
ohe_columns = list()
for lst in ohe.categories_:
    ohe_columns += lst.tolist()

In [ ]:
new_train_cat = pd.DataFrame(ohe.transform(tmp_train[cat_columns]), columns=ohe_columns)
new_valid_cat = pd.DataFrame(ohe.transform(tmp_valid[cat_columns]), columns=ohe_columns)
new_test_cat  = pd.DataFrame(ohe.transform(tmp_test[cat_columns]), columns=ohe_columns)

In [ ]:
new_train_cat.head()

In [ ]:
cat_columns

In [ ]:
tmp_train = pd.concat([tmp_train, new_train_cat], axis=1)
tmp_valid = pd.concat([tmp_valid, new_valid_cat], axis=1)
tmp_test = pd.concat([tmp_test, new_test_cat], axis=1)

# 기존 범주형 변수 제거
tmp_train = tmp_train.drop(columns=cat_columns)
tmp_valid = tmp_valid.drop(columns=cat_columns)
tmp_test = tmp_test.drop(columns=cat_columns)

In [ ]:
tmp_train.head()

In [ ]:
tmp_y_train = y_train
tmp_y_valid = y_valid

### 2. Scikit-Learn 분류 모델 사용해보기
Scikit-Learn의 기본 분류 모델을 사용해보겠습니다. <br>
각 모델의 평가 메트릭은 대회 평가 메트릭인 f1_score를 사용합니다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score

#### 1) 로지스틱 회귀 모델

In [ ]:
lr = LogisticRegression()

lr.fit(tmp_train, tmp_y_train)

y_pred = lr.predict(tmp_valid)

print(f"Logistic Regression F1 Score: {f1_score(tmp_y_valid, y_pred, average='micro')}")

#### 2) 서포트 벡터 머신(rbf 커널)

In [ ]:
svc = SVC()

svc.fit(tmp_train, tmp_y_train)

y_pred = svc.predict(tmp_valid)

print(f"Support Vector Machine F1 Score: {f1_score(tmp_y_valid, y_pred, average='micro')}")

#### 3) 랜덤 포레스트

In [ ]:
rf = RandomForestClassifier()

rf.fit(tmp_train, tmp_y_train)

y_pred = rf.predict(tmp_valid)

print(f"RandomForest F1 Score: {f1_score(tmp_y_valid, y_pred, average='micro')}")

#### 4) XGBoost

In [ ]:
xgb = XGBClassifier(tree_method='gpu_hist')

xgb.fit(tmp_train, tmp_y_train)

y_pred = xgb.predict(tmp_valid)

print(f"XGBoost F1 Score: {f1_score(tmp_y_valid, y_pred, average='micro')}")

#### 5) LightGBM

In [ ]:
lgb = LGBMClassifier(tree_method='gpu_hist')

lgb.fit(tmp_train, tmp_y_train)

y_pred = lgb.predict(tmp_valid)

print(f"LightGBM F1 Score: {f1_score(tmp_y_valid, y_pred, average='micro')}")

### 3. k-Fold Cross Validation
먼저 1. 에서 정리한 전처리 프로세스를 하나의 함수로 만듭니다.

In [ ]:
def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train = tmp_x_train.reset_index(drop=True)
    tmp_x_valid = tmp_x_valid.reset_index(drop=True)
    tmp_x_test  = tmp_x_test.reset_index(drop=True)
    
    tmp_x_train.loc[tmp_x_train[c] == '?', c] = tmp_x_train[c].mode()[0]
    tmp_x_valid.loc[tmp_x_valid[c] == '?', c] = tmp_x_valid[c].mode()[0]
    tmp_x_test.loc[tmp_x_test[c]   == '?', c] = tmp_x_test[c].mode()[0]
    
    tmp_x_train['log_capital_loss'] = tmp_x_train['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_valid['log_capital_loss'] = tmp_x_valid['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_test['log_capital_loss'] = tmp_x_test['capital_loss'].map(lambda x : np.log(x) if x != 0 else 0)
    
    tmp_x_train['log_capital_gain'] = tmp_x_train['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_valid['log_capital_gain'] = tmp_x_valid['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    tmp_x_test['log_capital_gain'] = tmp_x_test['capital_gain'].map(lambda x : np.log(x) if x != 0 else 0)
    
    tmp_x_train = tmp_x_train.drop(columns=['capital_loss', 'capital_gain'])
    tmp_x_valid = tmp_x_valid.drop(columns=['capital_loss', 'capital_gain'])
    tmp_x_test  = tmp_x_test.drop(columns=['capital_loss', 'capital_gain'])
    
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])
    
    tmp_all = pd.concat([tmp_x_train, tmp_x_valid, tmp_x_test])

    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_all[cat_columns])
    
    ohe_columns = list()
    for lst in ohe.categories_:
        ohe_columns += lst.tolist()
    
    tmp_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]), columns=ohe_columns)
    tmp_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]), columns=ohe_columns)
    tmp_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]), columns=ohe_columns)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_valid_cat], axis=1)
    tmp_x_test = pd.concat([tmp_x_test, tmp_test_cat], axis=1)

    tmp_x_train = tmp_x_train.drop(columns=cat_columns)
    tmp_x_valid = tmp_x_valid.drop(columns=cat_columns)
    tmp_x_test = tmp_x_test.drop(columns=cat_columns)
    
    return tmp_x_train.values, tmp_x_valid.values, tmp_x_test.values

In [ ]:
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) 
    return 'f1',f1_score(t, y_bin, average='micro')

In [ ]:
from sklearn.model_selection import StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0],))

for i, (trn_idx, val_idx) in enumerate(skf.split(train, label)):
    x_train, y_train = train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = train.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clf = XGBClassifier(tree_method='gpu_hist')
    
    # 모델 학습
    clf.fit(x_train, y_train,
            eval_set = [[x_valid, y_valid]], 
            eval_metric = xgb_f1,        
            early_stopping_rounds = 100,
            verbose = 100,  )

    # 훈련, 검증 데이터 Log Loss 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train), average='micro')
    val_f1_score = f1_score(y_valid, clf.predict(x_valid), average='micro')
    print('{} Fold, train f1_score : {:.4f}4, validation f1_score : {:.4f}\n'.format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    

# 교차 검증 F1 Score 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

### 4. OOF(Out-Of-Fold) 앙상블
k-Fold를 활용해서 모델 검증 및 각 폴드의 결과를 앙상블하는 OOF 앙상블 입니다.

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0], ))

for i, (trn_idx, val_idx) in enumerate(skf.split(train, label)):
    x_train, y_train = train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = train.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clf = XGBClassifier(tree_method='gpu_hist')
    
    # 모델 학습
    clf.fit(x_train, y_train,
            eval_set = [[x_valid, y_valid]], 
            eval_metric = xgb_f1,        
            early_stopping_rounds = 100,
            verbose = 100,  )

    # 훈련, 검증 데이터 F1 Score 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train), average='micro')
    val_f1_score = f1_score(y_valid, clf.predict(x_valid), average='micro')
    print('{} Fold, train f1_score : {:.4f}4, validation f1_score : {:.4f}\n'.format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    
    oof_pred += clf.predict_proba(x_test)[:, 1] / n_splits
    

# 교차 검증 F1 Score 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

### 5. Stacking 앙상블
2 stage 앙상블인 Stacking 앙상블 입니다. Stacking 앙상블은 수십개의 1 stage 모델의 결과를 모아 2 stage 모델로 학습 후 결과를 내는 앙상블 방식입니다.

#### 1) 1 stage 결과 모으기
Stacking 앙상블을 진행할 1 stage 모델의 결과(train, test)를 모읍니다. 

In [ ]:
val_scores = list()

new_x_train_list = [np.zeros((train.shape[0], 1)) for _ in range(4)]
new_x_test_list  = [np.zeros((test.shape[0], 1)) for _ in range(4)]

for i, (trn_idx, val_idx) in enumerate(skf.split(train, label)):
    print(f"Fold {i} Start")
    x_train, y_train = train.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = train.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clfs = [LogisticRegression(), 
            RandomForestClassifier(), 
            XGBClassifier(tree_method='gpu_hist'), 
            LGBMClassifier(tree_method='gpu_hist')]
    
    for model_idx, clf in enumerate(clfs):
        clf.fit(x_train, y_train)
        
        new_x_train_list[model_idx][val_idx, :] = clf.predict_proba(x_valid)[:, 1].reshape(-1, 1)
        new_x_test_list[model_idx][:] += clf.predict_proba(x_test)[:, 1].reshape(-1, 1) / n_splits

In [ ]:
new_x_train_list

In [ ]:
new_x_test_list

In [ ]:
new_train = pd.DataFrame(np.concatenate(new_x_train_list, axis=1), columns=None)
new_label = np.concatenate([tmp_y_train, tmp_y_valid])
new_test = pd.DataFrame(np.concatenate(new_x_test_list, axis=1), columns=None)

new_train.shape, new_label.shape, new_test.shape

#### 2) 2 Stage Meta Model 학습
new_train, new_test에 들어있는 변수는 모두 수치형 변수이므로 Standard Scaling만 진행하겠습니다.<br>
새로 생성한 데이터 new_train, new_test 데이터를 가지고 2 Stage Meta Model을 학습하고 결과를 만듭니다.

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0], ))

for i, (trn_idx, val_idx) in enumerate(skf.split(new_train, new_label)):
    x_train, y_train = new_train.iloc[trn_idx, :], new_label[trn_idx]
    x_valid, y_valid = new_train.iloc[val_idx, :], new_label[val_idx]
    
    # 전처리
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.transform(x_valid)
    x_test  = scaler.transform(new_test)
    
    # 모델 정의
    clf = XGBClassifier(tree_method='gpu_hist')
    
    # 모델 학습
    clf.fit(x_train, y_train,
            eval_set = [[x_valid, y_valid]], 
            eval_metric = xgb_f1,        
            early_stopping_rounds = 100,
            verbose = 100,  )

    # 훈련, 검증 데이터 F1 Score 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train), average='micro')
    val_f1_score = f1_score(y_valid, clf.predict(x_valid), average='micro')
    print('{} Fold, train f1_score : {:.4f}4, validation f1_score : {:.4f}\n'.format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    
#     oof_pred += clf.predict_proba(x_test)[:, 1] / n_splits
    

# 교차 검증 F1 Score 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

쌓은 모델이 적어서 성능이 좋지 않으니 OOF 앙상블로 예측한 값을 결과로 사용합니다. 

### 6. 결과 만들기

In [ ]:
submit = pd.read_csv("/kaggle/input/kakr-4th-competition/sample_submission.csv")

In [ ]:
submit.loc[:, 'prediction'] = (oof_pred > 0.5).astype(int)

In [ ]:
submit.head()

In [ ]:
submit.to_csv('submission.csv', index=False)